In [2]:
import matplotlib.pyplot as plt
import scipy.stats as stats
from pyddm import Model
from pyddm import Sample
import pyddm as ddm
import numpy as np
import pandas as pd
import random
import os
import copy

This is similar to the model recovery code, but with one participant with many trials, and a narrower range in the model fitting. I include this code because the original model recovery failed.

## Simulate the drift diffusion model

This is the most general model: it allows the upper and lower boundaries to collapse at different times and with different slopes, and for the drift rates for target presence and absence to have different means and standard deviations.

In [3]:
class SubjDDM:
    
    def __init__(self, subj_id, v, a, z, t0, tmax, sigma, collapse_time, collapse_rate, dt):
        """
        Initializes the drift diffusion model with the given parameters.

        Args:
        - subj_id: subject identifier
        - v: drift rates for target absence and presence
        - a: boundary
        - z: starting point bias (in absolute units, i.e. not scaled)
        - t0: non-decision time
        - tmax: maximum trial duration (not including t0)
        - sigma: standard deviations of the Gaussian noise for target absence and presence
        - collapse_time: the time in which the lower and upper boundaries start collapsing
        - collapse_rate: the speed with which the lower and upper boundaries collapse
        - dt: time step size
        """
        self.subj_id = subj_id
        self.v = v
        self.a = [-a,a] #lower and upper boundaries
        self.z = z
        self.t0 = t0
        self.tmax = tmax
        self.sigma = sigma
        self.collapse_time = collapse_time
        self.collapse_rate = collapse_rate
        self.dt = dt
    

    def simulate(self, n_trials):
        """
        Simulates the drift diffusion model for the given number of trials.

        Args:
        - n_trials: number of trials to simulate

        Returns:
        - A pandas DataFrame with columns "present", "decision", "rt" (response time) and "correct"
        """
        
        df = pd.DataFrame(columns=["subj_id", "present", "decision","rt", "correct"])

        for i in range(n_trials):
            
            present = random.randrange(2)
            v = self.v[present]
            s = self.sigma[present]
            a = copy.copy(self.a)            
            x = self.z
            t = 0.0
            
            while (a[0] < x < a[1]) & (t<self.tmax):
                
                x = x + v*self.dt + s*np.sqrt(self.dt)*random.normalvariate(0,1)
                
                if t>self.collapse_time[0]:
                    a[0]+=self.collapse_rate[0]*self.dt
                if t>self.collapse_time[1]:
                    a[1]+=self.collapse_rate[1]*self.dt
                    
                t = t + self.dt

            if (x >= a[1]) & (t<self.tmax-self.t0):
                decision = 1
                correct = decision==present
                rt = t + self.t0
                df.loc[i] = [self.subj_id, present,decision,rt, correct]
            elif (x<= a[0]) & (t<self.tmax-self.t0):
                decision = 0
                correct = decision==present
                rt = t + self.t0
                df.loc[i] = [self.subj_id, present,decision,rt, correct]
            else:
                continue
            
        return(df)
    
class GroupDDM:
    
    def __init__(self, v, a, z, t0, tmax, sigma, collapse_time, collapse_rate, dt):
        """
        Initializes the drift diffusion model with the given parameters.

        Args:
        - v: probability distributions for drift rates for target absence and presence
        - a: probability distributions for boundary
        - z: probability distributions for starting point bias (in absolute units, i.e. not scaled)
        - t0: probability distributions for non-decision time
        - tmax: maximum trial duration (not including t0)
        - sigma: probability distributions for standard deviations of the Gaussian noise for target absence and presence
        - collapse_time: probability distributions for the time in which the lower and upper boundaries start collapsing
        - collapse_rate: probability distributions for the speed with which the lower and upper boundaries collapse
        - dt: time step size
        """
        self.v = v
        self.a = a
        self.z = z
        self.t0 = t0
        self.tmax = tmax
        self.sigma = sigma
        self.collapse_time = collapse_time
        self.collapse_rate = collapse_rate
        self.dt = dt
        self.df = pd.DataFrame(columns=['subj_id', 'v_absent', 'v_present', 'a',
                                       'z', 't0', 'tmax', 'sigma_absent', 'sigma_present', 'ct_absent',
                                       'ct_present', 'cr_absent', 'cr_present', 'dt'])
        
    def simulate(self, n_subj, n_trials):
        """
        Simulates the drift diffusion model for the given number of subjects and trials.

        Args:
        - n_subj: number of subjects to simulate
        - n_trials: number of trials to simulate

        Returns:
        - A pandas DataFrame with columns "present", "decision", "rt" (response time) and "correct"
        """
        
        self.generated_data = pd.DataFrame()
        
        def sample_value(x):
            
            if (type(x)==float) | (type(x)==int):
                return x
            elif (type(x)==list):
                values = [];
                for a in x:
                    values.append(sample_value(a))
                return(values)
            else:
                return x.rvs()
            
            
        
        for i in range(n_subj):
            
            v_s = sample_value(self.v)
            a_s = sample_value(self.a)
            z_s = sample_value(self.z)
            t0_s = sample_value(self.t0)
            sigma_s = sample_value(self.sigma)
            collapse_time_s = sample_value(self.collapse_time)
            collapse_rate_s = sample_value(self.collapse_rate)
            
            self.df = self.df.append({'subj_id': i,
                            'v_absent': v_s[0], 
                            'v_present': v_s[1], 
                            'a': a_s, 
                            'z': z_s,
                            't0': t0_s,
                            'tmax': self.tmax, 
                            'sigma_absent': sigma_s[0], 
                            'sigma_present': sigma_s[1], 
                            'ct_absent': collapse_time_s[0],
                            'ct_present': collapse_time_s[1],
                            'cr_absent': collapse_rate_s[0], 
                            'cr_present': collapse_rate_s[1], 
                            'dt': self.dt}, ignore_index=True)
            
            subj = SubjDDM(i, v_s, a_s, z_s, t0_s, self.tmax, sigma_s, collapse_time_s, collapse_rate_s, self.dt)
            self.generated_data = pd.concat( [self.generated_data, subj.simulate(n_trials)])
            
            
            

In [6]:
random.seed(1)

Ntrials = 1000

m1 = SubjDDM(subj_id = 1,
             v=[-0.5,0.5],
               a = 1.5, 
               z = 0.2, 
               t0 = 0.3, 
               tmax =5, 
               sigma = [1,1],
               collapse_time = [0,0],
               collapse_rate = [0,0], 
               dt=0.01)
m1_df = m1.simulate(Ntrials)
m1_df.to_csv('data/generated_data/m1_one_subj_generated.csv', index=False)


In [30]:
random.seed(1)

#collapsing boundaries, drift rate for absence is 0
m2 = SubjDDM(subj_id = 1,
             v=[0,1],
               a = 1.5, 
               z = 0.2, 
               t0 = 0.3, 
               tmax =5, 
               sigma = [1,1],
               collapse_time = [0,0],
               collapse_rate = [0.2,-0.05], 
               dt=0.01)
m2_df = m2.simulate(Ntrials)
m2_df.to_csv('data/generated_data/m2_one_subj_generated.csv', index=False)

In [31]:
random.seed(1)

#Lower boundary starts collapsing rapidly after a fixed amount of time
m3 = SubjDDM(subj_id=1,
               v=[0,1],
               a = 2.5, 
               z = 0.2, 
               t0 = 0.3, 
               tmax =5, 
               sigma = [1,1],
               collapse_time = [1,0],
               collapse_rate = [6,0], 
               dt=0.01)
m3_df =  m3.simulate(Ntrials)
m3_df.to_csv('data/generated_data/m3_one_subj_generated.csv', index=False)

In [89]:
random.seed(1)

# noise can vary

m4 = SubjDDM(subj_id = 1,
             v=[-0.5,0.5],
               a = 1.5, 
               z = 0.2, 
               t0 = 0.3, 
               tmax =5, 
               sigma = [0.8,1],
               collapse_time = [0,0],
               collapse_rate = [0,0], 
               dt=0.01)
m4_df = m4.simulate(Ntrials)
m4_df.to_csv('data/generated_data/m4_one_subj_generated.csv', index=False)


In [90]:
random.seed(1)

# noise can vary and boundaries can collapse
m5 = SubjDDM(subj_id = 1,
             v=[0,1],
               a = 1.5, 
               z = 0.2, 
               t0 = 0.3, 
               tmax =5, 
               sigma = [0.8,1],
               collapse_time = [0,0],
               collapse_rate = [0.2,-0.05], 
               dt=0.01)
m5_df = m5.simulate(Ntrials)
m5_df.to_csv('data/generated_data/m5_one_subj_generated.csv', index=False)

In [91]:
random.seed(1)

# noise can vary and lower boundary starts collapsing rapidly after a fixed amount of time
m6 = SubjDDM(subj_id=1,
               v=[0,1],
               a = 2.5, 
               z = 0.2, 
               t0 = 0.3, 
               tmax =5, 
               sigma = [0.8,1],
               collapse_time = [1,0],
               collapse_rate = [6,0], 
               dt=0.01)
m6_df =  m6.simulate(Ntrials)
m6_df.to_csv('data/generated_data/m6_one_subj_generated.csv', index=False)

## Fit M1: basic model

In [8]:
from pyddm import InitialCondition,Fittable, Fitted
from pyddm.models import Drift, Bound, LossRobustBIC, NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture
from pyddm.functions import fit_adjust_model

class ICPointSideBiasRatio(InitialCondition):
    name = "A side-biased starting point expressed as a proportion of the distance between the bounds."
    required_parameters = ["x0"]
    required_conditions = ["present"]
    def get_IC(self, x, dx, conditions):
        x0 = self.x0/2 + .5 #rescale to between 0 and 1
        # Bias > .5 for left side correct, bias < .5 for right side correct.
        # On original scale, positive bias for left, negative for right
        if not conditions['present']==1:
            x0 = 1-x0
        shift_i = int((len(x)-1)*x0)
        assert shift_i >= 0 and shift_i < len(x), "Invalid initial conditions"
        pdf = np.zeros(len(x))
        pdf[shift_i] = 1. # Initial condition at x=x0*2*B.
        return pdf
    
class DriftPresent(ddm.models.Drift):
    name = "Drift depends on target presence"
    required_parameters = ["driftpresent","driftabsent"] # <-- Parameters we want to include in the model
    required_conditions = ["present"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, conditions, **kwargs):
        # so that driftabsent values are normally negative
        return self.driftpresent * conditions['present'] - self.driftabsent * (1-conditions['present'])




In [92]:
m1_spec = Model(name='Basic model drift varies as a function of target presence',
                 drift=DriftPresent(driftabsent=Fittable(minval=-1, maxval=0),
                                     driftpresent=Fittable(minval=0, maxval=1)),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundConstant(B=Fittable(minval=0.1, maxval=2.5)),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m1(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m1_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftabsent': [float(fit_model.parameters()['drift']['driftabsent'])],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m1/data_"+data_label):
                       os.makedirs("model_fits/model_m1/data_"+data_label)
                       
    with open("model_fits/model_m1/data_"+data_label+"/one_subj.txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)

for i in list(range(1,7)):
    
    loaded = pd.read_csv("data/generated_data/m"+str(i)+"_one_subj_generated.csv") 
    a= fit_m1(loaded.reset_index(),'m'+str(i))
    a.to_csv("model_fits/model_m1/data_m"+str(i)+"/one_subj.csv")

Info: Params [ 0.57490197 -0.60527069  1.37860372  0.1656827   0.47633015] gave 3440.4088189409213
Info: Params [ 0.94112066 -0.08897095  1.25125663  0.16962788  0.47144102] gave 3178.2220869421753
Info: Params [ 0.         -1.          0.94326426  0.15378898  0.95576406] gave 2271.7586911378476
Info: Params [ 0.58472457 -0.77241717  1.53845362  0.18015206  0.42894044] gave 3161.2168658797787
Info: Params [ 0.99727141 -0.17614112  1.41857124  0.22879943  0.4143022 ] gave 3270.077514822267
Info: Params [ 0.         -1.          1.00816849  0.14037249  0.93881913] gave 2145.003344867097


## Fit M2: boundaries can collapse

In [93]:
from pyddm import InitialCondition,Fittable, Fitted
from pyddm.models import Drift, Bound, LossRobustBIC, NoiseConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture, Noise
from pyddm.functions import fit_adjust_model

class BoundCollapse(Bound):
    name = "Boundary for the collapsing boundary model"
    required_parameters = ["B", "uppercollapse", "lowercollapse"]
    required_conditions = []
    def get_bound(self, t, conditions, **kwargs):
        return max(0.01, self.B + t*(self.uppercollapse-self.lowercollapse)/2)
    
class DriftPresentCollapse(ddm.models.Drift):
    name = "Drift depends on target presence and collapses"
    required_parameters = ["driftpresent","uppercollapse", "lowercollapse"] # <-- Parameters we want to include in the model
    required_conditions = ["present"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, t, conditions, **kwargs):
        # so that driftabsent values are normally negative
        return conditions['present'] * (self.driftpresent - (self.uppercollapse+self.lowercollapse)/2) + \
    (1-conditions['present']) *(0+ (self.uppercollapse+self.lowercollapse)/2)





In [94]:
uppercollapse = Fittable(minval=-1.5, maxval=0)
lowercollapse = Fittable(minval=0, maxval=1.5)

m2_spec = Model(name='Absence drift is zero boundaries can collapse',
                 drift=DriftPresentCollapse(driftpresent=Fittable(minval=0, maxval=1.5),
                                   uppercollapse=uppercollapse,
                                   lowercollapse=lowercollapse),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    uppercollapse=uppercollapse,
                                    lowercollapse=lowercollapse),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m2(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m2_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'uppercollapse': [float(fit_model.parameters()['drift']['uppercollapse'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m2/data_"+data_label):
                       os.makedirs("model_fits/model_m2/data_"+data_label)
                       
    with open("model_fits/model_m2/data_"+data_label+"/one_subj.txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

for i in list(range(1,7)):
    
    loaded = pd.read_csv("data/generated_data/m"+str(i)+"_one_subj_generated.csv") 
    a= fit_m2(loaded.reset_index(),'m'+str(i))
    a.to_csv("model_fits/model_m2/data_m"+str(i)+"/one_subj.csv")

Info: Params [0.82070597 0.         0.4        1.69045502 0.03058873 0.21018856] gave 3488.3601043071185
Info: Params [0.97123184 0.         0.2        1.36663306 0.21552465 0.48366614] gave 3149.4934376242595
Info: Params [ 0.30655039 -0.07500001  1.5         1.35448295  0.21234119  0.92713713] gave 1652.0921471813044
Info: Params [0.98223545 0.         0.8        2.46971586 0.08521243 0.04465192] gave 3195.3823826107964
Info: Params [ 1.17371081 -0.05952241  0.44047759  1.85666802  0.22140294  0.39232361] gave 3167.2637790559706
Info: Params [ 0.41012499 -0.06000001  1.5         1.35293959  0.15750648  0.90430241] gave 1545.4864048127918


## Fit M3: Lower boundary collapses rapidly


In [105]:
class BoundNonlinearCollapse(Bound):
    name = "Lower boundary collapses nonlinearly"
    required_parameters = ["B", "lowercollapse", "collapsetime"]
    required_conditions = []
    def get_bound(self, t, conditions, **kwargs):
        if t<self.collapsetime:
            return(self.B)
        else:
            return max(0.01, self.B - (t-self.collapsetime)*self.lowercollapse/2)
    
class DriftPresentNonlinearCollapse(ddm.models.Drift):
    name = "Drift depends on target presence and collapses nonlinearly"
    required_parameters = ["driftpresent","lowercollapse", "collapsetime"] # <-- Parameters we want to include in the model
    required_conditions = ["present"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, t, conditions, **kwargs):
        if t<self.collapsetime:
            return conditions['present'] * (self.driftpresent)
        else:
            return conditions['present'] * (self.driftpresent-self.lowercollapse/2) + \
        (1-conditions['present']) * (0+ self.lowercollapse/2)



In [96]:
lowercollapse = Fittable(minval=0, maxval=10)
collapsetime = Fittable(minval=0.5, maxval=2.5)

m3_spec = Model(name='Absence drift is zero lower boundary can collapse nonlinearly',
                 drift=DriftPresentNonlinearCollapse(driftpresent=Fittable(minval=0, maxval=1.5),
                                   lowercollapse=lowercollapse,
                                   collapsetime=collapsetime),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundNonlinearCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    lowercollapse=lowercollapse,
                                    collapsetime=collapsetime),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m3(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m3_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'collapsetime': [float(fit_model.parameters()['drift']['collapsetime'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m3/data_"+data_label):
                       os.makedirs("model_fits/model_m3/data_"+data_label)
                       
    with open("model_fits/model_m3/data_"+data_label+"/one_subj.txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

for i in list(range(1,7)):
    
    loaded = pd.read_csv("data/generated_data/m"+str(i)+"_one_subj_generated.csv") 
    a= fit_m3(loaded.reset_index(),'m'+str(i))
    a.to_csv("model_fits/model_m3/data_m"+str(i)+"/one_subj.csv")

Info: Params [ 0.87195924  0.49694708  0.79862112  1.64073558 -0.05879735  0.2634765 ] gave 3483.146279430003
Info: Params [1.03312116 0.31869765 0.78622297 1.36384496 0.16562696 0.45848337] gave 3141.241691538097
Info: Params [0.77093543 3.70060796 0.57304312 1.61405974 0.05086619 0.75691114] gave 1230.9406697661238
Info: Params [ 1.00800321  0.74193686  0.55652352  2.15686704 -0.06136193  0.08366961] gave 3194.0678677001724
Info: Params [1.16413004 0.49002816 0.55479823 1.7403265  0.18425895 0.30461878] gave 3164.9648298458983
Info: Params [0.97030391 4.19999899 0.57619042 1.75708537 0.01111543 0.73393667] gave 1032.975527829098


## Fit M4: unequal variance

In [97]:
class NoiseAsymmetric(Noise):
    name = "Noise is higher when a target is present"
    required_parameters = ["noiseabsent"]
    required_conditions = ['present']
    def get_noise(self, t, conditions, **kwargs):
        if conditions['present']==1:
            return 1
        else:
            return self.noiseabsent

In [100]:
m4_spec = Model(name='Basic model drift varies as a function of target presence',
                 drift=DriftPresent(driftabsent=Fittable(minval=-1, maxval=0),
                                     driftpresent=Fittable(minval=0, maxval=1)),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0.5,maxval=2)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundConstant(B=Fittable(minval=0.1, maxval=2.5)),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m4(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m4_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftabsent': [float(fit_model.parameters()['drift']['driftabsent'])],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m4/data_"+data_label):
                       os.makedirs("model_fits/model_m4/data_"+data_label)
                       
    with open("model_fits/model_m4/data_"+data_label+"/one_subj.txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)

for i in list(range(1,7)):
    
    loaded = pd.read_csv("data/generated_data/m"+str(i)+"_one_subj_generated.csv") 
    a= fit_m4(loaded.reset_index(),'m'+str(i))
    a.to_csv("model_fits/model_m4/data_m"+str(i)+"/one_subj.csv")

Info: Params [ 0.5438881  -0.6139136   0.95581912  1.35199569  0.19095974  0.42072455] gave 3444.7838058522534
Info: Params [ 1.         -0.07055144  1.09753294  1.36107694  0.13983053  0.4957331 ] gave 3185.439879317282
Info: Params [ 0.         -1.          0.66279578  0.82958968  0.18927555  0.92068082] gave 2179.6973031367156
Info: Params [ 0.49224572 -0.70942882  0.78133734  1.29243471  0.21897114  0.54344606] gave 3118.3904610414065
Info: Params [ 0.92570966 -0.12388459  0.830476    1.22171321  0.18132346  0.41588978] gave 3254.5818023176585
Info: Params [ 0.         -1.          0.5         0.83233794  0.06548259  0.96517852] gave 1850.7158498579693


## Fit M5: unequal variance + collapsing boundaries

In [103]:
m5_spec = Model(name='Absence drift is zero boundaries can collapse',
                 drift=DriftPresentCollapse(driftpresent=Fittable(minval=0, maxval=1.5),
                                   uppercollapse=uppercollapse,
                                   lowercollapse=lowercollapse),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0.5,maxval=2)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    uppercollapse=uppercollapse,
                                    lowercollapse=lowercollapse),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m5(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m5_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'uppercollapse': [float(fit_model.parameters()['drift']['uppercollapse'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m5/data_"+data_label):
                       os.makedirs("model_fits/model_m5/data_"+data_label)
                       
    with open("model_fits/model_m5/data_"+data_label+"/one_subj.txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

for i in list(range(1,7)):
    
    loaded = pd.read_csv("data/generated_data/m"+str(i)+"_one_subj_generated.csv") 
    a= fit_m5(loaded.reset_index(),'m'+str(i))
    a.to_csv("model_fits/model_m5/data_m"+str(i)+"/one_subj.csv")

Info: Params [0.86616068 0.         0.45714279 0.92472747 1.73785311 0.01254172
 0.2330222 ] gave 3481.0804002620653
Info: Params [1.05825955 0.         0.29230769 1.05361398 1.50666011 0.21954026
 0.49391071] gave 3153.3259735685283
Info: Params [ 0.67327181 -0.00358254  2.99692139  0.87791278  2.07088125  0.38889645
  0.83952883] gave 1387.8121970499055
Info: Params [0.98556684 0.         0.72499995 0.77127472 2.20828451 0.00485542
 0.06599888] gave 3146.812797296273
Info: Params [ 1.00451763 -0.10647693  0.36018974  0.76783972  1.57527106  0.17747251
  0.34543286] gave 3129.0104729656173
Info: Params [0.85915694 0.         2.97500251 0.69395729 2.04000469 0.30878223
 0.85810657] gave 1152.0910725420956


## Fit M6: unequal variance + nonlinearly collapsing boundary

In [106]:
m6_spec = Model(name='Absence drift is zero lower boundary can collapse nonlinearly',
                 drift=DriftPresentNonlinearCollapse(driftpresent=Fittable(minval=0, maxval=1.5),
                                   lowercollapse=lowercollapse,
                                   collapsetime=collapsetime),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0.5,maxval=2)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundNonlinearCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    lowercollapse=lowercollapse,
                                    collapsetime=collapsetime),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m6(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m6_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'collapsetime': [float(fit_model.parameters()['drift']['collapsetime'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m6/data_"+data_label):
                       os.makedirs("model_fits/model_m6/data_"+data_label)
                       
    with open("model_fits/model_m6/data_"+data_label+"/one_subj.txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

for i in list(range(1,7)):
    
    loaded = pd.read_csv("data/generated_data/m"+str(i)+"_one_subj_generated.csv") 
    a= fit_m6(loaded.reset_index(),'m'+str(i))
    a.to_csv("model_fits/model_m6/data_m"+str(i)+"/one_subj.csv")

Info: Params [ 0.85791426  0.4857139   0.79999829  0.92597723  1.5882872  -0.07187085
  0.21285685] gave 3485.130498709798
Info: Params [1.02844611 0.31999534 0.78749544 1.0444078  1.40693649 0.18792395
 0.44455816] gave 3148.4644025536395
Info: Params [ 0.76246004  3.64999755  0.5684928   0.94464698  1.39549257 -0.02590064
  0.86393631] gave 1235.4129356019776
Info: Params [ 1.00792709  0.6914282   0.5000001   0.77770015  1.93603079 -0.11376964
  0.15275898] gave 3146.798748949068
Info: Params [1.0413953  0.40000028 0.69999998 0.74897778 1.30763384 0.1393067
 0.4895413 ] gave 3137.7920131574438
Info: Params [ 0.92624583  3.66463227  0.5688522   0.73327004  1.58085211 -0.04299708
  0.75031251] gave 994.9381183394074


In [18]:
from pyddm import FitResult
from pyddm import Fittable, Fitted
import pyddm.plot


def plotOneSubj(model_fit, model_gen):
    
    with open("model_fits/model_"+model_fit+"/data_"+model_gen+"/one_subj.txt", "r") as f:
        model = eval(f.read())
        
    data_loaded = pd.read_csv("data/generated_data/"+model_gen+"_one_subj_generated.csv") 

    pyddm.plot.model_gui(model=model, sample = Sample.from_pandas_dataframe(data_loaded, rt_column_name="rt", correct_column_name="correct"))


In [108]:
plotOneSubj('m4','m1')